In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

In [34]:
df_train = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/train.csv",encoding="ISO-8859-1")
df_test = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/test.csv",encoding="ISO-8859-1")
df_desc = pd.read_csv("C:/Users/JayZheng/Documents/Data Science/data mining/home-depot/data/product_descriptions.csv")

In [48]:
num_trainrow = df_train.shape[0]

In [36]:
df_test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [32]:
df_desc.head(5)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [39]:
stemmer = SnowballStemmer('english')

In [37]:
def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())


In [38]:

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

df_all = pd.merge(df_all, df_desc, how='left', on='product_uid')


In [40]:
#start feature engineering
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))

df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']

In [43]:
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [49]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)

ValueError: labels ['search_term' 'product_title' 'product_description' 'product_info'] not contained in axis

In [51]:
df_train = df_all.iloc[:num_trainrow]
df_test = df_all.iloc[num_trainrow:]
test_id = df_test['id']

In [57]:
Y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [58]:
df_all.head()

,id,product_uid,relevance,len_of_query,word_in_title,word_in_description
0,2,100001,3.00,2,1,1
1,3,100001,2.50,2,1,1
2,9,100002,3.00,2,1,1
3,16,100005,2.33,3,1,1
4,17,100005,2.67,3,3,2


In [69]:
%timeit
fit = RandomForestRegressor(n_estimators=10, max_depth=6, random_state=0)
model = BaggingRegressor(fit, n_estimators=45, max_samples=0.1, random_state=25)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [74]:
y_pred.view()

array([ 2.05722814,  2.22731707,  2.22731707, ...,  2.1122656 ,
        2.41677289,  2.31755636])

In [76]:
pd.DataFrame({"id": test_id, "relevance": y_pred}).to_csv('randomforest.csv',index=False)